<a href="https://colab.research.google.com/github/shihmujan/classs/blob/main/Alzheimer_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**讀取資料**

In [1]:
import gdown
import zipfile 
import os 

def zip_list(file_path):
    zf = zipfile.ZipFile(file_path, 'r')
    zf.extractall()

url = 'https://drive.google.com/u/2/uc?id=1Pc4yDR0c-WrClg7_PmzggqwUbATTxJAH&export=download&confirm=t'
output = "Alzheimer.zip"
gdown.download(url, output)

file_path = "Alzheimer.zip"
zip_list(file_path)

Downloading...
From: https://drive.google.com/u/2/uc?id=1Pc4yDR0c-WrClg7_PmzggqwUbATTxJAH&export=download&confirm=t
To: /content/Alzheimer.zip
100%|██████████| 456M/456M [00:05<00:00, 88.7MB/s]


KeyboardInterrupt: ignored

In [ ]:
!pip install lime
!pip install einops

**插入套件**

In [ ]:
import os
from Load_Dataset import *
from tqdm import tqdm
from util import *
from trainer import trainer
from torchvision.models.resnet import resnet34
from Models import *
import lime
from lime import lime_image
import random
from skimage.segmentation import mark_boundaries

**預設訓練學習率**

In [ ]:
lr = 0.001  #學習率
resume = True  #是否使用預訓練資料
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
epoch = 1000
batch_size = 25
momentum = 0.9

**訓練資料**

In [ ]:
train_ds, test_ds, idx_to_class = Return_Dataset(batch_size) #設定data loader
model = coatnet_2().to(device) #設定模型

if resume: 
    # Load checkpoint.
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('checkpoint/ckpt.pth')
    model.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch']

optimizer = torch.optim.Adam(
    params=model.parameters(),
    lr=lr,
    betas=(0.9, 0.999),
)
loss_function = nn.CrossEntropyLoss()
trainer = trainer(
    model=model,
    optimizer=optimizer,
    loss_function=loss_function,
    epoch=epoch, 
    resume=resume,
    trainloader=train_ds,
    testloader=test_ds,
    device=device
)
#trainer.test_epoch(0) #測試準確度

**工具箱初始化**

In [ ]:
tool = Utils(model,idx_to_class) 

**獲取影像與標籤**

In [ ]:
dataset = ImageFolder('Alzheimer/train')
valset = ImageFolder('Alzheimer/test')

In [ ]:
img,lab,pred = tool.get_test_img(dataset) #獲取圖片、標籤、預測結果
tool.is_the_same(lab,pred) #print 預測是否正確

**顯示解釋器對影像的前處理**

In [ ]:
explainer = lime_image.LimeImageExplainer()
exp = explainer.explain_instance(img, tool.show_batch_predict, top_labels=1, hide_color=0,num_samples=10)

**重新獲取影像、標籤與預測結果**

In [ ]:
img,lab,pred = tool.get_test_img(dataset)
tool.is_the_same(lab,pred)

**放入解釋器解釋**

In [ ]:
explainer = lime_image.LimeImageExplainer()
exp = explainer.explain_instance(img, tool.batch_predict, top_labels=4, hide_color=0,num_samples=200)

In [ ]:
dict_heatmap = dict(exp.local_exp[lab])
heatmap = np.vectorize(dict_heatmap.get)(exp.segments) 
print('label:',idx_to_class[lab],'predict:',idx_to_class[pred])
plt.imshow(img)
plt.show()
plt.imshow(heatmap, cmap = 'RdBu', vmin  = -heatmap.max(), vmax = heatmap.max())
plt.colorbar()

In [ ]:
# 只顯示重要部分
temp, mask = exp.get_image_and_mask(lab, positive_only=True, num_features=1, hide_rest=True)
img = mark_boundaries(temp / 255.0, mask)
plt.imshow(img)

In [ ]:
#顯示重要部分
temp, mask = exp.get_image_and_mask(lab, positive_only=True, num_features=1, hide_rest=False)
img = mark_boundaries(temp / 255.0, mask)
plt.imshow(img)

In [ ]:
#增加遮罩式意圖
temp, mask = exp.get_image_and_mask(lab, positive_only=False, num_features=1, hide_rest=False)
img = mark_boundaries(temp / 255.0, mask)
plt.imshow(img)

In [ ]:
#對於其他類別分類結果
temp, mask = exp.get_image_and_mask(lab, positive_only=False, num_features=4, hide_rest=False)
img = mark_boundaries(temp / 255.0, mask)
plt.imshow(img)

不同類別的解釋結果->每一列為一個類別

search_correct 只尋找正確預測結果

search_error 只尋錯誤預測結果

search_random 隨機尋找正確與錯誤

[0,0,0,0] 尋找標籤為0的類別

[1,2,3,4] 標籤為1234各找一個 **粗體文字**

In [ ]:
import cv2
img_list = tool.search_correct([0,1,2,3],dataset)
tool.run_all_image_list_and_show(img_list)

In [ ]:
img_list = tool.search_correct([0,0,0,0],dataset)
tool.run_all_image_list_and_show(img_list)

In [ ]:
img_list = tool.search_correct([1,1,1,1],dataset)
tool.run_all_image_list_and_show(img_list)

In [ ]:
img_list = tool.search_correct([2,2,2,2],dataset)
tool.run_all_image_list_and_show(img_list)

In [ ]:
img_list = tool.search_correct([3,3,3,3],dataset)
tool.run_all_image_list_and_show(img_list)

In [ ]:
img_list = tool.search_error([0,1,2,3],valset)
tool.run_all_image_list_and_show(img_list)